In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm, trange
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
import librosa
import torch
import laion_clap
# from extraction.vgg_sound import *
# 在aws notebook环境需要加上这个
import sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "..")))

from audioldm import image_to_audio, build_model, clap_to_audio
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import soundfile as sf 





/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:

from models import *
from vgg_dataset import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = V2AMapperMLPImproved().to(device)

In [5]:
# mde

In [6]:
ckpt_path = "ckpts/best_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# model = V2AMapperMLP()
model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
model = model.to(device)

In [8]:
MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)

Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense

In [9]:

class LargeVideoTestDataset(Dataset):
    def __init__(self, data_dir, subset_ratio = 0.2, transform=None):
        """
        root_dir: 保存所有 .pth 文件的目录，每个文件对应一个 sample。
        transform: 如果需要对数据做预处理，可在这里传入。
        """
        super().__init__()
        # 仅收集当前目录下所有的 pth 文件列表
        file_list = []

        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".pth"):
                    file_list.append(os.path.join(root, file))

        # 仅使用前 20% 的数据
        num_samples = int(len(file_list) * subset_ratio)

        self.file_paths = sorted(file_list)[:num_samples]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 在这里按需读取，而不是一次性加载全部
        pth_path = self.file_paths[idx]
        sample_data = torch.load(pth_path)  
        clip_feat = sample_data['clip_features']  # (64, 512)
        clap_feat = sample_data['clap_features']  # (1, 512)
        id = sample_data['id']
        caption = sample_data['caption']

        if self.transform:
            clip_feat, clap_feat = self.transform((clip_feat, clap_feat))

        return clip_feat, clap_feat, id, caption

In [22]:
root = "/mnt/new_volume2/vgg_sound_emb"
partition = "train"
data_dir = f"{root}/{partition}"
data_dir ="/mnt/new_volume2/ai_video_emb/"
vgg_sound = LargeVideoTestDataset(data_dir, subset_ratio = 1)

In [23]:
from tqdm import trange

In [24]:
# error = []

# for i in trange(len(vgg_sound)):
#     try:
#         clip_feat, clap_feat, id, caption = vgg_sound[i]
#     except Exception as e:
#         error.append(vgg_sound.file_paths[i])
#         print(vgg_sound.file_paths[i])

In [25]:
# error

In [25]:
val_ratio = 0.1
test_ratio = 0.1

total_len = len(vgg_sound)
val_len = int(total_len * val_ratio)
test_len = int(total_len * test_ratio)
train_len = total_len - val_len - test_len
train_dataset, val_dataset, test_dataset = random_split(
    vgg_sound, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42), 
)


In [26]:
test_loader = DataLoader(
    vgg_sound, 
    batch_size=1, 
    shuffle=False, 
    num_workers=2,
    pin_memory=True
)

In [27]:

def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = clap_to_audio(
        latent_diffusion=audioldm,
        clap_feat=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        # filename = f"{file_name}.wav"
        filename = file_name
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

In [28]:
video_path = "/mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video/"

In [29]:
cnt = 0
for clip_feat, clap_feat, id, caption in tqdm(test_loader):
    # clip_feat: (1, 64, 512)
    # clap_feat: (1, 512)
    # id: (1,)
    # caption: (1,)
    print(clip_feat.shape, clap_feat.shape, id, caption)
    clip_feat = clip_feat.to(device)  # (64, 512)
    clap_feat = clap_feat.squeeze(0).to(device)  # (512)

    gen_clap_feat = model(clip_feat).to(device=device)  # (1, 512)
    # calculate cosine similarity
    cos = nn.CosineSimilarity(dim=0)  # compare whole vectors
    cos_sim = cos(gen_clap_feat.squeeze(0), clap_feat.squeeze(0).squeeze(0))  # both are (512,)

    print("cosine similarity:", cos_sim)
    # print(cos_sim.shape)
    # print(cos_sim)
    image2audio(
        file_name="/home/ubuntu/project/v2a-mapper/train/test_inf/test_ag/test_outputs/" + id[0][:-4] + "_gen.wav",
        images=gen_clap_feat.to(device=device),
        duration=10,
        guidance_scale=2.5,
        random_seed=42,
        n_candidates=3
    )
    # !ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0][:-4]}.wav

    cnt += 1
    if cnt == 20:
        break

  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['01jrxxj2c9fa59dw655kktte42.mp4'] ['-']
cosine similarity: tensor(0.3410, device='cuda:0', grad_fn=<SumBackward1>)


 50%|█████     | 1/2 [00:18<00:18, 18.03s/it]

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['piano.mp4'] ['-']
cosine similarity: tensor(0.3440, device='cuda:0', grad_fn=<SumBackward1>)


100%|██████████| 2/2 [00:35<00:00, 18.00s/it]


In [30]:
cnt = 0
for clip_feat, clap_feat, id, caption in tqdm(test_loader):
    # clip_feat: (1, 64, 512)
    # clap_feat: (1, 512)
    # id: (1,)
    # caption: (1,)
    print(clip_feat.shape, clap_feat.shape, id, caption)
    clip_feat = clip_feat.to(device)  # (64, 512)
    clap_feat = clap_feat.squeeze(0).to(device)  # (512)

    gen_clap_feat = model(clip_feat).to(device=device)  # (1, 512)
    # calculate cosine similarity
    cos = nn.CosineSimilarity(dim=0)  # compare whole vectors
    cos_sim = cos(gen_clap_feat.squeeze(0), clap_feat.squeeze(0).squeeze(0))  # both are (512,)

    print("cosine similarity:", cos_sim)
    # print(cos_sim.shape)
    # print(cos_sim)
    image2audio(
        file_name="/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/" + id[0][:-4] + "_gen.wav",
        images=gen_clap_feat.to(device=device),
        duration=10,
        guidance_scale=2.5,
        random_seed=42,
        n_candidates=3
    )
    !ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0][:-4]}.wav

    cnt += 1
    if cnt == 20:
        break

  0%|          | 0/12783 [00:00<?, ?it/s]

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['Hkb8B7Xi3hI_000030.mp4'] ['female singing']
cosine similarity: tensor(0.2452, device='cuda:0', grad_fn=<SumBackward1>)
key: fbank
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
<class 'torch.Tensor'>
torch.Size([1, 1, 1024, 64])


DDIM Sampler: 100%|██████████| 200/200 [00:18<00:00, 11.09it/s]
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/home/ubuntu/miniconda3/envs/v2a-mapper --cc=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-l

  0%|          | 1/12783 [00:23<82:07:29, 23.13s/it]

...

  0%|          | 19/12783 [05:54<65:13:32, 18.40s/it]

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['YXFcsZYysSE_000010.mp4'] ['fireworks banging']
cosine similarity: tensor(0.8835, device='cuda:0', grad_fn=<SumBackward1>)
key: fbank
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
<class 'torch.Tensor'>
torch.Size([1, 1, 1024, 64])


DDIM Sampler: 100%|██████████| 200/200 [00:17<00:00, 11.50it/s]


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/home/ubuntu/miniconda3/envs/v2a-mapper --cc=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-l

  0%|          | 19/12783 [06:12<69:33:37, 19.62s/it]
